In [1]:
import pandas as pd
import numpy as np

In [2]:
from pycaret.regression import *

In [3]:
import mlflow

In [4]:
data = pd.read_csv("raw.csv")
data.head()

,Posted On,BHK,Rent,Size,Floor,Area Type,Area Locality,City,Furnishing Status,Tenant Preferred,Bathroom,Point of Contact
0,2022-05-18,2,10000,1100,Ground out of 2,Super Area,Bandel,Kolkata,Unfurnished,Bachelors/Family,2,Contact Owner
1,2022-05-13,2,20000,800,1 out of 3,Super Area,"Phool Bagan, Kankurgachi",Kolkata,Semi-Furnished,Bachelors/Family,1,Contact Owner
2,2022-05-16,2,17000,1000,1 out of 3,Super Area,Salt Lake City Sector 2,Kolkata,Semi-Furnished,Bachelors/Family,1,Contact Owner
3,2022-07-04,2,10000,800,1 out of 2,Super Area,Dumdum Park,Kolkata,Unfurnished,Bachelors/Family,1,Contact Owner
4,2022-05-09,2,7500,850,1 out of 2,Carpet Area,South Dum Dum,Kolkata,Unfurnished,Bachelors,1,Contact Owner


In [5]:
# mlflow.set_tracking_uri('http://127.0.0.1:5000')

In [18]:
mlflow.set_tracking_uri('http://ec2-54-212-194-129.us-west-2.compute.amazonaws.com:5000') 

In [26]:
s = setup(data, 
          target='Rent', 
          session_id=123, 
          experiment_name='experiment_et_tunned', 
          log_experiment=True)

,Description,Value
0,Session id,123
1,Target,Rent
2,Target type,Regression
3,Original data shape,"(4746, 12)"
4,Transformed data shape,"(4746, 23)"
5,Transformed train set shape,"(3322, 23)"
6,Transformed test set shape,"(1424, 23)"
7,Ordinal features,1
8,Numeric features,3
9,Categorical features,8


2023/07/01 12:35:18 INFO mlflow.tracking.fluent: Experiment with name 'experiment_et_tunned' does not exist. Creating a new experiment.


In [23]:
# best = compare_models()

In [27]:
best = create_model('et')

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,17230.3321,3385564672.0912,58185.6054,0.5246,0.5355,0.5040
1,24802.7027,37019872933.0150,192405.4909,0.0476,0.5919,0.5125
2,13179.3193,760092689.5878,27569.7786,0.6245,0.5463,0.5369
3,15064.3198,1609608364.7782,40119.9248,0.4055,0.5447,0.5176
4,21312.0037,5671222919.9750,75307.5223,-0.5848,0.6209,0.7683
5,17633.8230,2451763819.7642,49515.2887,0.4744,0.5579,0.5015
6,12899.9405,823682194.8450,28699.8640,0.7141,0.5435,0.5523
7,12562.3062,857307053.8160,29279.8062,0.5723,0.5553,0.5641
8,13304.2569,1886766480.6530,43436.9253,0.4708,0.5413,0.4952


In [ ]:
# check the residuals of trained model
plot_model(best, plot = 'residuals_interactive')

In [ ]:
# check feature importance
plot_model(best, plot = 'feature')

In [ ]:
# model1 = create_model('rf')

In [28]:
tune = tune_model(best, n_iter = 10)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,16209.6611,3681630670.0546,60676.4425,0.4831,0.4933,0.4772
1,23117.4277,36445685725.6479,190907.5319,0.0623,0.5459,0.4993
2,9744.3693,377414291.9425,19427.1535,0.8136,0.4422,0.4412
3,12796.6179,1017381115.1333,31896.4123,0.6242,0.4630,0.4603
4,12844.6648,988093480.1905,31433.9543,0.7239,0.4887,0.5058
5,15317.4663,1855384177.4528,43074.1707,0.6023,0.4814,0.4585
6,10758.9957,801116656.3942,28304.0042,0.7220,0.4544,0.4617
7,10168.4092,480421252.8764,21918.5139,0.7603,0.4254,0.4285
8,12118.2576,1612279875.7315,40153.2050,0.5478,0.4881,0.4420


Fitting 10 folds for each of 10 candidates, totalling 100 fits


In [29]:
final = finalize_model(best)

In [ ]:
#

In [29]:
# save pipeline
save_model(final, 'model')

Transformation Pipeline and Model Successfully Saved


(Pipeline(memory=FastMemory(location=C:\Users\CHRIST~1\AppData\Local\Temp\joblib),
          steps=[('numerical_imputer',
                  TransformerWrapper(include=['BHK', 'Size', 'Bathroom'],
                                     transformer=SimpleImputer())),
                 ('categorical_imputer',
                  TransformerWrapper(include=['Posted On', 'Floor', 'Area Type',
                                              'Area Locality', 'City',
                                              'Furnishing Status',
                                              'Tenant Preferred',
                                              'Point of Contact'],...
                                                               handle_missing='return_nan',
                                                               use_cat_names=True))),
                 ('rest_encoding',
                  TransformerWrapper(include=['Posted On', 'Floor',
                                              'Area Localit

In [ ]:
!mlflow ui

In [17]:
deploy_model(final, model_name = 'model', 
             platform = 'aws', 
             authentication = { 'bucket'  : 'ml-model-001' })

Model Successfully Deployed on AWS S3


In [18]:
help(deploy_model)

Help on function deploy_model in module pycaret.regression.functional:

deploy_model(model, model_name: str, authentication: dict, platform: str = 'aws')
    This function deploys the transformation pipeline and trained model on cloud.
    
    
    Example
    -------
    >>> from pycaret.datasets import get_data
    >>> boston = get_data('boston')
    >>> from pycaret.regression import *
    >>> exp_name = setup(data = boston,  target = 'medv')
    >>> lr = create_model('lr')
    >>> # sets appropriate credentials for the platform as environment variables
    >>> import os
    >>> os.environ["AWS_ACCESS_KEY_ID"] = str("foo")
    >>> os.environ["AWS_SECRET_ACCESS_KEY"] = str("bar")
    >>> deploy_model(model = lr, model_name = 'lr-for-deployment', platform = 'aws', authentication = {'bucket' : 'S3-bucket-name'})
    
    
    Amazon Web Service (AWS) users:
        To deploy a model on AWS S3 ('aws'), the credentials have to be passed. The easiest way is to use environment
        var